# CK0223 - Mineração de Dados

## Lista 02 - Extração de Dados

### Dados do discente:
**Nome**: Luiza Esther Martins Pessoa
**Matrícula**: 555516

### Vídeo Youtube:

---

### **(a)** Ler o dataset *fakeTelegram.BR_2022.csv*

Como realizado na [Lista 01 - Tratamento de Dados](https://github.com/EstherMart/Data-Mining/blob/main/Lista01_TratamentoDeDados/tratamento.ipynb), ler o dataset é o primeiro passo para iniciarmos a extraçãa, manipulação e tratamento dos dados. 

Para fazer isso, começamos importando as bibliotecas necessárias para leitura (`pandas`) e para download local da base de dados (`gdown`). Além disso, vale ressaltar que existem outras formas de realizar o upload para o repositório local, mas decidi seguir a lógica de puxar e realizar o download de base utilizando apenas o link disponibilizado pelo professor.

**Importante**: Alguns trechos de código serão reutilizados, visto que são as mesmas exigências em ambas as listas e tal solicitação já foi resolvida anteriormente.

In [1]:
# IMPORTAÇÃO DE BIBLIOTECAS
import gdown
import pandas as pd
import requests
from io import StringIO

Fazendo o download do dataset para o repositório local

In [2]:
origem_url = 'https://drive.google.com/file/d/1c_hLzk85pYw-huHSnFYZM_gn-dUsYRDm/view'

# O ID do arquivo (necessário para fazer o download direto) está entre os últimos elementos da URL.
# Fazemos um split na URL usando '/' como separador e pegamos o penúltimo elemento da lista.
# Isso funciona porque a estrutura da URL é:
# https://drive.google.com/file/d/ID_DO_ARQUIVO/view
# E ao aplicar url.split('/'), o resultado será:
# ['https:', '', 'drive.google.com', 'file', 'd', 'ID_DO_ARQUIVO', 'view?...']
# Portanto, o ID está na posição -2 (penúltima).

file_id = origem_url.split('/')[-2]

# URL do arquivo no formato aceito pelo gdown
url = f'https://drive.google.com/uc?id={file_id}'

# Nome local do arquivo que será baixado
output = 'fakeTelegram.BR_2022.csv'

# Baixando o arquivo com gdown
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1c_hLzk85pYw-huHSnFYZM_gn-dUsYRDm
From (redirected): https://drive.google.com/uc?id=1c_hLzk85pYw-huHSnFYZM_gn-dUsYRDm&confirm=t&uuid=e1e74e96-6cfc-4915-abe2-f990edb6cf7b
To: c:\Users\esthe\Downloads\Data-Mining\Lista02_ExtracaoDeDados\fakeTelegram.BR_2022.csv
100%|██████████| 224M/224M [01:14<00:00, 3.02MB/s] 


'fakeTelegram.BR_2022.csv'

Leitura do dataset utilizando o método `.read_csv()`, pois permite carregar dados estruturados a partir de arquivos *CSV* para um DataFrame do pandas, sendo uma estrutura tabular extremamente versátil para análise de dados e visualmente intuitiva.

In [3]:
df_inicial = pd.read_csv("fakeTelegram.BR_2022.csv")

Os códigos abaixo foram implementados apenas para fins de **visualização e verificação inicial dos dados carregados**, com o objetivo de:
- Obter uma compreensão rápida da dimensão do dataset
- Identificar os nomes das colunas disponíveis
- Realizar uma primeira checagem da integridade básica da estrutura de dados

In [4]:
# PARA MELHOR VISUALIZAÇÃO
print("Número de linhas:", df_inicial.shape[0])
print("Número de colunas:", df_inicial.shape[1])
print("\nColunas disponíveis:")
for i, col in enumerate(df_inicial.columns, 1):
    print(f"{i}. {col}")

Número de linhas: 557586
Número de colunas: 19

Colunas disponíveis:
1. date_message
2. id_member_anonymous
3. id_group_anonymous
4. media
5. media_type
6. media_url
7. has_media
8. has_media_url
9. trava_zap
10. text_content_anonymous
11. dataset_info_id
12. date_system
13. score_sentiment
14. score_misinformation
15. id_message
16. message_type
17. messenger
18. media_name
19. media_md5


O método `display()` do IPython renderiza objetos como DataFrames de forma mais rica que print(), mantendo a formatação tabular. Combinado com `.style`, permite personalizar a aparência: `.set_caption()` adiciona títulos descritivos, enquanto `.set_properties()` aplica estilos CSS como cores de fundo suaves (#f8f9fa), bordas sutis, truncamento de texto longo (text-overflow: ellipsis) e limitação de largura (max-width), melhorando a legibilidade e organização dos dados exibidos.

Essa abordagem foi utilizada, pois é importante ter em mente as primeiras e últimas amostras do dataset.

In [5]:
# VISUALIZANDO AMOSTRAS INICIAIS E FINAIS DO DATASET
display(df_inicial.head().style.set_caption("Primeiros Registros").set_properties(**{
    'background-color': '#f8f9fa',
    'border': '1px solid #dee2e6',
    'color': '#212529',
    'max-width': '300px',
    'overflow': 'hidden',
    'text-overflow': 'ellipsis',
    'white-space': 'nowrap'
}))

display(df_inicial.tail().style.set_caption("Últimos Registros").set_properties(**{
    'background-color': '#f8f9fa',
    'border': '1px solid #dee2e6',
    'color': '#212529',
    'max-width': '300px',
    'overflow': 'hidden',
    'text-overflow': 'ellipsis',
    'white-space': 'nowrap'
}))

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,nan,nan,nan,False,False,False,Então é Fato Renato o áudio que eu ouvi no whatsapp isso ocorreu em Niterói principalmente no bairro Fonseca ?,5,2022-10-05 06:25:28.863641,0.000000,nan,16385,Texto,telegram,nan,nan
1,2022-10-05 06:25:08,nan,12283e08a2eb5789201e105b34489ee7,nan,nan,nan,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás, infelizmente não consigo enviar para cá, mas é facilmente verificável no YouTube do presidente",5,2022-10-05 06:25:28.926311,0.064400,nan,16386,Texto,telegram,nan,nan
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,nan,nan,nan,False,False,False,"É isso, nossa parte já foi quase toda feita. No segundo turno completamos nossa parte desse teatro. Essa é uma guerra de 4* geração na dimensão humana e uma guerra espiritual do bem contra o mal na dimensão do Universo. Pensamento positivo é fundamental, pensem sempre em algo bom. Deus continua nos abençoando, nosso livre arbítrio completa o curso.",5,2022-10-05 06:26:29.361949,-0.355100,0.157242,16366,Texto,telegram,nan,nan
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,nan,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.000000,nan,19281,Imagem,telegram,nan,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,nan,True,False,False,nan,5,2022-10-05 06:28:29.316325,nan,nan,507185,Imagem,telegram,nan,5ad5c8bbe9da93a37fecf3e5aa5b0637


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-tem-lucro-de-r-96-bilhoes-no-terceiro-trimestre/,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro trimestre ☛ https://terrabrasilnoticias.com/2022/11/bndes-tem-lucro-de-r-96-bilhoes-no-terceiro-trimestre/",5,2022-11-16 14:49:39.146502,0.102700,nan,575796,Url,telegram,nan,25e43b6a58b848c43ad5b5f9e979822a
557582,2022-11-11 12:09:08,nan,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,https://youtu.be/8g1Vz9_0xVk,5,2022-11-16 14:49:39.847434,0.000000,nan,1286443,Url,telegram,nan,657949d03e4088f6b332e2686ccd3221
557583,2022-11-11 12:09:47,nan,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro. https://t.me/vemprasruas",5,2022-11-16 14:49:39.922279,0.000000,nan,13294,Imagem,telegram,nan,a21848a61045380a6483866daed0ca0e
557584,2022-11-11 12:09:46,nan,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro. https://t.me/vemprasruas",5,2022-11-16 14:49:39.992932,0.000000,nan,1286444,Imagem,telegram,nan,a21848a61045380a6483866daed0ca0e
557585,2022-11-11 12:09:48,nan,b11f2df64ac19aad47a50accf32052d6,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro. https://t.me/vemprasruas",5,2022-11-16 14:49:40.064006,0.000000,nan,192127,Imagem,telegram,nan,a21848a61045380a6483866daed0ca0e


---

### **(b)** Remova os trava-zaps.

Este item será resolvido através da manipulação da coluna `trava_zap`, previamente identificada na análise inicial. A estratégia adotada considera o tamanho do dataset (557.586 registros) e segue um fluxo estruturado:

1. **Análise Preliminar**
    - Verificação do tipo de dado (dtype) e valores únicos na coluna `trava_zap`
    - Confirmação da proporção True/False/NaN (se aplicável)

In [6]:
# ANÁLISE PRELIMINAR
# Verificação do tipo e valores únicos
print(f"Tipo de dado: {df_inicial['trava_zap'].dtype}")
trava_dist = df_inicial['trava_zap'].value_counts(dropna=False)
display(trava_dist.to_frame().style.set_caption("Contagem de valores distintos em trava_zap"))

# Verificação de linhas que devem ser removidas
print(f"Registros com trava_zap que precisam ser removidos: {trava_dist.get(True, 0):,}")

Tipo de dado: bool


,count
trava_zap,
False,557570
True,16


Registros com trava_zap que precisam ser removidos: 16


2. **Processamento por Chunks**
    - Divisão do dataset em blocos de 1.000 linhas para:
        - Otimização de memória
        - Facilidade de debug
        - Monitoramento do progresso

In [7]:
import numpy as np

In [8]:
# Parâmetros do chunk -- 1000 linhas 
CHUNK_SIZE = 1000
total_chunks = (len(df_inicial) // CHUNK_SIZE) + 1
df_sem_trava_zap = pd.DataFrame() # Criação de DataFrame Seguro para controle de variáveis.

print(f"\033[1mProcessando {total_chunks} chunks de {CHUNK_SIZE} registros cada:\033[0m")

for i, chunk in enumerate(np.array_split(df_inicial, total_chunks)):
    # Filtro principal -- linhas que possuem trava zap, isto é, trava_zap = True
    chunk_filtrado = chunk[chunk['trava_zap'] != True]
    
    # Concatenção segura -- recebendo apenas os blocos onde trava_zap = False
    df_sem_trava_zap = pd.concat([df_sem_trava_zap, chunk_filtrado], ignore_index=True)
    
    # Log de progresso para controle da análise
    if (i+1) % 10 == 0:
        print(f"Processado chunk {i+1}/{total_chunks} | Registros retidos: {len(df_sem_trava_zap):,}")

Processando 558 chunks de 1000 registros cada:


c:\Users\esthe\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Processado chunk 10/558 | Registros retidos: 10,000
Processado chunk 20/558 | Registros retidos: 20,000
Processado chunk 30/558 | Registros retidos: 29,999
Processado chunk 40/558 | Registros retidos: 39,999
Processado chunk 50/558 | Registros retidos: 49,999
Processado chunk 60/558 | Registros retidos: 59,999
Processado chunk 70/558 | Registros retidos: 69,999
Processado chunk 80/558 | Registros retidos: 79,999
Processado chunk 90/558 | Registros retidos: 89,998
Processado chunk 100/558 | Registros retidos: 99,998
Processado chunk 110/558 | Registros retidos: 109,998
Processado chunk 120/558 | Registros retidos: 119,998
Processado chunk 130/558 | Registros retidos: 129,998
Processado chunk 140/558 | Registros retidos: 139,998
Processado chunk 150/558 | Registros retidos: 149,992
Processado chunk 160/558 | Registros retidos: 159,982
Processado chunk 170/558 | Registros retidos: 169,972
Processado chunk 180/558 | Registros retidos: 179,962
Processado chunk 190/558 | Registros retidos: 1

4. **Saída Controlada**
    - Relatório pós-processamento mostrará:
        - Número total de linhas removidas
        - Porcentagem impactada
        - Amostra das primeiras 5 linhas removidas (se relevante)

In [9]:
# Relatório final
print("\033[1mRelatório de Remoção:\033[0m")
print(f"• Registros originais: {len(df_inicial):,}")
print(f"• Registros removidos: {trava_dist.get(True, 0):,}")
print(f"• Registros restantes: {len(df_sem_trava_zap):,}") 

# Visualização dos dados que foram removidos
if trava_dist.get(True, 0) > 0:
    display(
        df_inicial[df_inicial['trava_zap'] == True].head(16)  
    )

Relatório de Remoção:
• Registros originais: 557,586
• Registros removidos: 16
• Registros restantes: 557,570


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
21944,2022-10-07 07:46:52,NaN,c712c1b704c22bd0cef50bc06125cdbd,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-07 07:47:00.355052,0.0000,0.067344,53260,Texto,telegram,NaN,NaN
89109,2022-10-16 00:45:02,8a30ac374bc4b5930eaf0667a178546a,e56ec342fc599ebb4ed89655eb6f03aa,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-16 00:45:04.064662,0.0000,0.056698,521324,Texto,telegram,NaN,NaN
294541,2022-10-04 14:22:47,39ee10516124280a22f1798f2a41f9a7,959f13e0079883060632c74ffc81c547,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-04 14:22:48.808572,0.9734,0.010433,27241,Texto,telegram,NaN,NaN
324567,2022-10-25 14:55:55,e003fbb6ffedb1838e42360d41cab314,5b10d7739171149be6d9961e3350c071,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-25 14:56:11.604972,0.9460,0.403945,1182938,Texto,telegram,NaN,NaN
389164,2022-10-30 20:19:52,NaN,c8f2de56550ed0bf85249608b7ead93d,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-30 20:19:54.183578,0.0000,NaN,28330,Texto,telegram,NaN,NaN
423083,2022-11-03 00:40:23,NaN,4d3712f5a117e36180d4b4cbd07c540e,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-03 00:40:48.993091,0.0000,0.067026,171104,Texto,telegram,NaN,NaN
466735,2022-11-07 20:47:35,4a498818da925377eff2606a260cfa45,f61777908059b318385882ff47b15c33,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-07 20:48:16.653321,0.0000,0.036173,88890,Texto,telegram,NaN,NaN
467297,2022-11-08 10:07:14,4a498818da925377eff2606a260cfa45,f61777908059b318385882ff47b15c33,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-08 10:07:25.260375,0.0000,0.072637,88977,Texto,telegram,NaN,NaN
471273,2022-11-08 20:37:52,4a498818da925377eff2606a260cfa45,f61777908059b318385882ff47b15c33,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-08 20:38:02.773746,0.0000,0.031973,89039,Texto,telegram,NaN,NaN
478270,2022-11-09 19:50:12,4a498818da925377eff2606a260cfa45,f61777908059b318385882ff47b15c33,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-09 19:50:36.039894,0.0000,0.029651,89212,Texto,telegram,NaN,NaN


---

### **(c)** Exportar os dados para um arquivo Parquet. 

Existe um método do pandas chamado `to_parquet`, sendo a forma mais eficiente de exportar DataFrames para o formato Parquet. Sua sintaxe básica inclui dois parâmetros essenciais:

```bash
df.to_parquet(
    path='dados.parquet',  # Caminho do arquivo de saída
    engine='auto'         # Motor de processamento (pyarrow/fastparquet)
)
```

**Observação**: O `engine='auto'` deixa o Pandas escolher automaticamente o motor disponível (geralmente PyArrow ou Fastparquet), mas eu costumo usar o pyarrow explicitamente porque ele é mais estável e tem suporte a mais funcionalidades. Além disso, ele permite leitura e escrita super rápidas, e mantém os tipos de dados de forma mais fiel.

In [10]:
df_sem_trava_zap.to_parquet('fakeTelegram.BR_2022_filtrado.parquet', engine='pyarrow')

---

### **(d)** Exportar os dados para o DuckDB. 

Nesta etapa, exportei o DataFrame anterior para um banco de dados DuckDB, que é um sistema de gerenciamento de banco de dados analítico leve e embutido. Com isso, realizei alguns passos importantes.

Primeiramente, fiz a instalação e a importação do DuckDB:

In [11]:
# !pip install duckdb -- Apenas se ainda não estiver instalado
import duckdb

Criação (ou conexão) com o banco DuckDB:

In [12]:
conn = duckdb.connect("fakeTelegram.BR_2022_dados.duckdb") 

Exportação do `df_sem_trava_zap` para uma tabela do banco:

In [13]:
conn.register("df_sem_trava_zap", df_sem_trava_zap)  # Registra o DataFrame como uma "tabela temporária"
conn.execute("CREATE TABLE IF NOT EXISTS tabela_dados AS SELECT * FROM df_sem_trava_zap")  # Exporta os dados

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Verificação dos dados no banco:

In [14]:
conn.execute("SELECT * FROM tabela_dados LIMIT 5").fetchdf()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,None,None,None,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,None,None
1,2022-10-05 06:25:08,None,12283e08a2eb5789201e105b34489ee7,None,None,None,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,None,None
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,None,None,None,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,None,None
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,None,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,None,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,None,True,False,False,None,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,None,5ad5c8bbe9da93a37fecf3e5aa5b0637


Como os próximos itens são de recuperação de dados utilizando o DuckDB, não vamos encerrar a conexão neste item, mas sim no final do processo.

---

### **(e)** Utlizando o DuckDB recupere:

#### **1**. A quantidade de mensagens

Para resolver este item, podemos utilizar de consultas SQL. A primeira consulta realizada foi para contar o total de mensagens armazenadas na tabela `tabela_dados` do banco DuckDB. Cada linha da tabela representa uma mensagem única, enviada em grupos do Telegram relacionados ao contexto do dataset analisado.
> A tabela `tabela_dados` contém **557.570 mensagens** e **19 atributos**, entre eles: `date_message`, `id_group_anonymous`, `has_media`, `trava_zap`, `text_content_anonymous`, entre outros que serão úteis nas análises a seguir.

In [39]:
qtd_mensagem = conn.execute("SELECT COUNT(*) AS total_mensagens FROM tabela_dados").fetchdf()
display(qtd_mensagem)

,total_mensagens
0,557570


Esta consulta realiza uma contagem multidimensional das mensagens, fornecendo diferentes perspectivas sobre a composição do dataset:

In [41]:
# text_content_anonymous leva em consideração apenas mensagem de texto
qtd_mensagem_adicional = conn.execute("""
    SELECT 
    COUNT(id_message) AS mensagens_com_id_valido,
    COUNT(DISTINCT id_message) AS mensagens_unicas,
    SUM(CASE WHEN text_content_anonymous IS NOT NULL THEN 1 ELSE 0 END) AS mensagens_textuais,
    SUM(CASE WHEN has_media = TRUE THEN 1 ELSE 0 END) AS mensagens_com_midia
    FROM tabela_dados
""").fetchdf()

display(qtd_mensagem_adicional)

,mensagens_com_id_valido,mensagens_unicas,mensagens_textuais,mensagens_com_midia
0,557570,364000,444201.0,332604.0


---

#### **2**. A quantidade de usuários

Para recuperar a quantidade de usuários únicos no dataset, utilizamos a coluna `id_member_anonymous`, que contém identificadores anônimos dos membros que enviaram mensagens nos grupos do Telegram. A consulta SQL no DuckDB foi a seguinte:

In [33]:
# Consulta para contar usuários únicos
qtd_usuarios = conn.execute("""
    SELECT 
    COUNT(DISTINCT id_member_anonymous) AS total_usuarios_unicos,
    COUNT(*) AS total_registros,
    ROUND(COUNT(DISTINCT id_member_anonymous) * 100.0 / COUNT(*), 2) AS percentual_identificados
    FROM tabela_dados
""").fetchdf()

display(qtd_usuarios)

,total_usuarios_unicos,total_registros,percentual_identificados
0,14809,557570,2.66


**Observações importantes**

- Algumas mensagens possuem `id_member_anonymous` como `NULL`, indicando que o remetente é anônimo ou não identificado.

- O número de usuários únicos é menor que o total de mensagens, pois um mesmo usuário pode ter enviado múltiplas mensagens.

In [34]:
qtd_sem_id = conn.execute("""
    SELECT 
    COUNT(CASE WHEN id_member_anonymous IS NULL THEN 1 END) AS mensagens_anonimas,
    COUNT(CASE WHEN id_member_anonymous IS NOT NULL THEN 1 END) AS mensagens_identificadas,
    COUNT(DISTINCT id_member_anonymous) AS usuarios_unicos_identificados
FROM tabela_dados
""").fetchdf()

display(qtd_sem_id)

,mensagens_anonimas,mensagens_identificadas,usuarios_unicos_identificados
0,323337,234233,14809


**Relação quantitativa:**

- Total de mensagens não nulas: 444.201

- Usuários únicos: 14.809

- Mensagens sem identificação: 323.337

---

#### **3**. A quantidade de grupos

Para identificar a quantidade de grupos únicos no dataset, utilizamos a coluna `id_group_anonymous` que contém identificadores únicos dos grupos do Telegram.

In [43]:
qtd_grupos = conn.execute("SELECT COUNT(DISTINCT id_group_anonymous) AS total_grupos_unicos FROM tabela_dados").fetchdf()
display(qtd_grupos)

,total_grupos_unicos
0,178


Sobre a coluna `id_group_anonymous`:

- **Valores**: São hashes MD5 únicos para cada grupo.
- **NULL**: Indica grupos não identificados (raro em seu dataset).

Relação com outras métricas:

- Cada grupo contém múltiplos `id_member_anonymous` (membros).
- Está relacionado com `message_type` (tipo de conteúdo predominante no grupo).

Dados técnicos:

- **Cardinalidade esperada**: Muito menor que o total de mensagens.
- **Tipo de dado**: TEXT (32 caracteres hexadecimais).

In [45]:
distribuicao_grupos = conn.execute("""
    SELECT 
    COUNT(DISTINCT id_group_anonymous) AS grupos_unicos,
    COUNT(*) AS total_mensagens,
    ROUND(COUNT(*) * 1.0 / COUNT(DISTINCT id_group_anonymous), 2) AS media_mensagens_por_grupo
    FROM tabela_dados                                                        
""").fetchdf()

display(distribuicao_grupos)

,grupos_unicos,total_mensagens,media_mensagens_por_grupo
0,178,557570,3132.42


---

#### **4**. Quantidade de mensagens que possuem apenas texto

Esta métrica identifica mensagens **exclusivamente textuais**, ou seja, que não contêm nenhum tipo de mídia anexada. Portanto, utilizei dados de duas colunas: `text_content_anonymous` e `has_media = FALSE`.

```sql
SELECT
    COUNT(*) AS mensagens_apenas_texto
FROM tabela_dados
WHERE 
    text_content_anonymous IS NOT NULL  -- Possui conteúdo textual
    AND has_media = FALSE               -- Não possui mídia anexada

In [ ]:
qtd_mensagem_texto = conn.execute("""
    SELECT
    COUNT(*) AS total_mensagens,
    SUM(CASE WHEN text_content_anonymous IS NOT NULL AND has_media = FALSE THEN 1 ELSE 0 END) AS mensagens_apenas_texto,
    ROUND(SUM(CASE WHEN text_content_anonymous IS NOT NULL AND has_media = FALSE THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS percentual_apenas_texto
    FROM tabela_dados                             
""").fetchdf()

display(qtd_mensagem_texto)

,total_mensagens,mensagens_apenas_texto,percentual_apenas_texto
0,557570,224966.0,40.35


#### **5**. Quantidade de mensagens contendo mídias

In [ ]:
# SUM(CASE WHEN text_content_anonymous IS NULL AND has_media = TRUE THEN 1 ELSE 0 END) AS mensagens_apenas_midia,
# SUM(CASE WHEN text_content_anonymous IS NOT NULL AND has_media = TRUE THEN 1 ELSE 0 END) AS mensagens_mistas